In [ ]:
import requests, json, concurrent, pandas as pd
from bs4 import BeautifulSoup

site_categories = []

main_page_content = requests.get(f'https://panorama.pub').content
bs_instance = BeautifulSoup(main_page_content, 'html.parser')
useless_categories_links = ['/', '/books', '/donate', '/about']
for nav_link in bs_instance.find_all('a', class_='nav-link'):
  if nav_link['href'] in useless_categories_links:
    continue
  site_categories.append({
      'link': nav_link['href'],
      'name': nav_link.text.replace('\n', '').strip()
  })

parsed_news = {site_category['name']: [] for site_category in site_categories}
def get_pages(category, pages_amount, page_num):
  print(f'Скачиваю страницу номер {page_num} из {pages_amount}')
  page_content = requests.get(f'https://panorama.pub{category["link"]}?page={page_num}').content

  for news_link in BeautifulSoup(page_content, 'html.parser').find_all('a', class_='entry big'):
    news_content = requests.get(f'https://panorama.pub{news_link["href"]}').content
    bs_instance = BeautifulSoup(news_content, 'html.parser')

    news_obj = {
        'title': bs_instance.find('h1').text.replace('\n', '').strip(),
        'text': ' '.join([tag.text for tag in bs_instance.find_all('p')]) 
    }
    parsed_news[category['name']].append(news_obj)
    print(news_obj['title'])


for category in site_categories:
  print('#' * 20)
  print(f'Скачиваю категорию: {category["name"]}')
  category_content = requests.get(f'https://panorama.pub{category["link"]}').content

  bs_instance = BeautifulSoup(category_content, 'html.parser')
  pages_amount = int(bs_instance.find('ul', class_='pagination').find_all('li')[-2].select('a')[0].text)
  with concurrent.futures.ThreadPoolExecutor() as executor: # optimally defined number of threads
    res = [executor.submit(get_pages, category, pages_amount, page_num) for page_num in range(1, pages_amount + 1)]
    concurrent.futures.wait(res)
      
    # with open('/content/dump_articles.json', 'w') as f:
    #   f.write(json.dumps(parsed_news))
  print('#' * 20)

data_list = []
for category in site_categories:
  for news in parsed_news[category['name']]:
    data_list.append({
      'Заголовок': news['title'],
      'Текст': news['text'],
       'Категория': category['name']
    })
pd.DataFrame(data_list)

In [ ]:
pd.DataFrame(data_list).to_csv('/content/articles.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv('/content/articles.csv').head()

In [ ]:
df['Текст'][0]

'Более 4 тысяч берлинцев приняли участие в народных гуляньях по случаю годовщины выписки российского блогера Алексея Навального из клиники «Шарите». Помимо украинской и грузинской диаспор, в спонтанных празднованиях приняли участие сотрудники американского посольства. Несмотря на то, что гулянья не были санкционированы властями Берлина, полиция не стала разгонять их водомётами и обеспечивала безопасность вблизи площади Маккейна, где собрались участники. Берлинцы организовали там несколько стихийных митингов в поддержку освобождения Навального из заключения. Их участники также требовали усилить санкционное давление на российские власти и ускорить процесс вступления Украины и Грузии в НАТО. Также на площади звучала музыка, выступали ансамбли самодеятельности, продавали матрёшки и другие сувениры. Группа волонтёров организовала сбор пожертвований в поддержку российских политзаключённых. В финале мероприятия перед протестующими выступила замгоссекретаря США Виктория Нуланд, поблагодарившая